In [1]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("Hugging Face")

login(token = hf_token)



The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
%pip install git+https://github.com/huggingface/transformers
%pip install langchain chromadb pypdf openai sentence-transformers accelerate langchain-community python-docx

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-m5yqnlpb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-m5yqnlpb
  Resolved https://github.com/huggingface/transformers to commit 1082361a1978d30db5c3932d1ee08914d74d9697
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.43.0.dev0-py3-none-any.whl size=9345100 sha256=718f6c5fe9d6b131fbd8820b161c3054a6d3dbbedfaf12c260f15b2b8fae3221
  Stored in directory: /tmp/pip-ephem-wheel-cache-e3jf_wpt/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
Note: you may need to restart the kernel

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import torch

# model_kwargs = {'device': 'cuda'}
# embeddings = HuggingFaceEmbeddings(model_kwargs=model_kwargs)

# embeddings = HuggingFaceEmbeddings(
#     model_name="sentence-transformers/all-MiniLM-L6-v2",
#     model_kwargs={'device': 'cpu'})

# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
# model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True,)

# pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300, temperature = 0.01)
# llm = HuggingFacePipeline(pipeline=pipe)

import pandas as pd
import pathlib
import docx
from langchain.docstore.document import Document
import os
import time

def read_docx(file_path):
    doc = docx.Document(file_path)
    return "\n".join([paragraph.text for paragraph in doc.paragraphs])

def load_all_files(directory_path):
    data = []
    for file_path in pathlib.Path(directory_path).glob("*"):
        if file_path.suffix == '.csv':
            df = pd.read_csv(file_path)
            for _, row in df.iterrows():
                content = " ".join(str(value) for value in row.values)
                data.append(Document(page_content=content))
        elif file_path.suffix == '.txt':
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                data.append(Document(page_content=content))
        elif file_path.suffix == '.docx':
            content = read_docx(file_path)
            data.append(Document(page_content=content))
        elif file_path.suffix == '.xlsx':
            df = pd.read_excel(file_path)
            for _, row in df.iterrows():
                content = " ".join(str(value) for value in row.values)
                data.append(Document(page_content=content))
    return data

# documents = load_all_files('D:\Axis-FAQ-chatbot\Data')


def interpret_files(documents):
    print(f"Total documents loaded: {len(documents)}")
    splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    texts = splitter.split_documents(documents)
    # print(f"Total texts generated: {len(texts)}")
    return texts

def create_embeddings():
    print("Creating embeddings")
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cuda'}
    )
    return embeddings

def save(texts, embeddings):
    print("Saving data")
    # Store data into database
    db=Chroma.from_documents(texts,embedding=embeddings,persist_directory="test_index")
    db.persist()
    # db = FAISS.from_documents(texts, embeddings)
    # db.save_local("faiss")


def load_llm(model_name):
    print("Loading LLM")
    if model_name == "phi3":
        print("Loading Phi3 model")
        tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
        model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", device_map='auto', torch_dtype="auto", trust_remote_code=True,)

    elif model_name == "llama":
        print("Loading Llama model")
        tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
        model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", device_map='auto', torch_dtype=torch.float16, trust_remote_code=True,)
        
    elif model_name == "gemma":
        print("Loading gemma model")
        tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b-it")
        model = AutoModelForCausalLM.from_pretrained("google/gemma-2-9b-it", device_map='auto', torch_dtype=torch.fp8e5m2, trust_remote_code=True,)

    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)
    llm = HuggingFacePipeline(pipeline=pipe)
    return llm


def retrieve_docs(embeddings, llm):
    print("Retrieving documents")

    # Load the database
    vectordb = Chroma(persist_directory="test_index", embedding_function = embeddings)

    # Load the retriver
    retriever = vectordb.as_retriever(search_kwargs = {"k" : 2})

    print("Retrieved documents")

    qna_prompt_template= """Use the following pieces of information to answer the user's question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    {context}
    Question: {question}
    Answer:"""

    PROMPT = PromptTemplate(
       template=qna_prompt_template, input_variables=["context","question"] # Change 'Context' to 'context' and 'Question' to 'question'
    )

    print("Sending the chain")
    # Define the QNA chain
    chain = RetrievalQA.from_chain_type(llm=llm,
                                             chain_type='stuff',
                                             retriever=retriever,
                                             return_source_documents=True,
                                             chain_type_kwargs={'prompt': PROMPT})

    if chain:
        print("Chain created")
    return chain







    # template = """Use the following pieces of information to answer the user's question.
    # If you don't know the answer, just say that you don't know, don't try to make up an answer.
    # {context}
    # Question: {question}
    # Answer:"""

    # db = Chroma(persist_directory="test_index", embedding_function = embeddings)
    
    # retriever = db.as_retriever(search_kwargs={'k': 2})
    # prompt = PromptTemplate(
    # template=template, input_variables=["context","question"] # Change 'Context' to 'context' and 'Question' to 'question'
    # )

    # print("Sending the chain")
    # # Define the QNA chain
    # QA_LLM = RetrievalQA.from_chain_type(llm=llm,
    #                                         chain_type='stuff',
    #                                         retriever=retriever,
    #                                         return_source_documents=True,
    #                                         chain_type_kwargs={'prompt': prompt})
    # return QA_LLM

# # Load the database
# vectordb = Chroma(persist_directory="test_index", embedding_function = embeddings)

# # Load the retriver
# retriever = vectordb.as_retriever(search_kwargs = {"k" : 3})

# qna_prompt_template= """Use the following pieces of information to answer the user's question.
# If you don't know the answer, just say that you don't know, don't try to make up an answer.
# {context}
# Question: {question}
# Answer:"""

# PROMPT = PromptTemplate(
#    template=qna_prompt_template, input_variables=["context","question"] # Change 'Context' to 'context' and 'Question' to 'question'
# )

# # Define the QNA chain
# chain = RetrievalQA.from_chain_type(llm=llm,
#                                          chain_type='stuff',
#                                          retriever=retriever,
#                                          return_source_documents=True,
#                                          chain_type_kwargs={'prompt': PROMPT})


def answer_question(chain, question):
    time_start = time.time()
    output = chain({'query': question})
    response = output["result"]
    time_elapsed = time.time() - time_start
    print(f'response time: {time_elapsed:.02f} sec')
  
    if "Answer:" in response:
        response = response.split("Answer:")[1].strip()

    response_lines = response.split('\n')
    cleaned_response = " ".join(line.strip() for line in response_lines if not line.startswith("Question:") and not line.startswith("Document:"))

    answer = cleaned_response.strip()
    return response

# question = input("Please enter your question: ")
# answer = answer_question(question,QA_LLM)



if __name__ == "__main__":
    data_path = "/kaggle/input/axisfaq-data"
    documents = load_all_files(data_path)
    texts = interpret_files(documents)
    embeddings = create_embeddings()
    save(texts, embeddings)

    # model_path = "D:\Axis-FAQ-chatbot\models\llama-2-7b-chat.ggmlv3.q8_0.bin"
    model_name = input("Enter the model name (phi3 or llama or gemma): ")
    llm = load_llm(model_name)
    QA_LLM = retrieve_docs(embeddings, llm)

#     user_input = input("What is your question? \n")
#     answer = answer_question(QA_LLM, user_input)
#     print(f"Answer: {answer}")

    while True:
        user_input = input("\n What is your question? \n")
        if user_input.lower() == "quit":
            break
        answer = answer_question(QA_LLM, user_input)
        print(f"Answer: {answer}")

        continue_input = input("\n Do you want to ask another question? (yes to continue, quit to exit, change to switch model): ").strip().lower()
        if continue_input == "quit":
            break
        elif continue_input == "change":
            model_name = input("\n Enter the new model name (phi3 or llama or gemma): ")
            llm = load_llm(model_name)
            QA_LLM = retrieve_docs(embeddings, llm)

Total documents loaded: 51
Creating embeddings
Saving data


Enter the model name (phi3 or llama or gemma):  phi3


Loading LLM
Loading Phi3 model


tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

Retrieving documents
Retrieved documents
Sending the chain
Chain created



 What is your question? 
  How to open a Savings Account


response time: 27.40 sec
Answer: Customers can open a savings account using the Mobile Banking app or internet banking with a few simple clicks.



 Do you want to ask another question? (yes to continue, quit to exit, change to switch model):  yes

 What is your question? 
  Documents required for locker facility


response time: 87.75 sec
Answer: A locker agreement needs to be submitted at the Branch for availing the Locker facility on a notarized stamp paper. This will be provided to you by the Branch and Stamp charges including GST will be debited from your Savings Account with us basis your consent on the Debit Slip or on issuance of a cheque for this amount. Please carry 2 passport size photograph to be affixed in agreement.


# Llama Responses

In [ ]:
if __name__ == "__main__":
    data_path = "/kaggle/input/axisfaq-data"
    documents = load_all_files(data_path)
    texts = interpret_files(documents)
    embeddings = create_embeddings()
    save(texts, embeddings)

    # model_path = "D:\Axis-FAQ-chatbot\models\llama-2-7b-chat.ggmlv3.q8_0.bin"
    model_name = input("Enter the model name (phi3 or llama or gemma): ")
    llm = load_llm(model_name)
    QA_LLM = retrieve_docs(embeddings, llm)

#     user_input = input("What is your question? \n")
#     answer = answer_question(QA_LLM, user_input)
#     print(f"Answer: {answer}")

    while True:
        user_input = input("\n What is your question? \n")
        if user_input.lower() == "quit":
            break
        answer = answer_question(QA_LLM, user_input)
        print(f"Answer: {answer}")

        continue_input = input("\n Do you want to ask another question? (yes to continue, quit to exit, change to switch model): ").strip().lower()
        if continue_input == "quit":
            break
        elif continue_input == "change":
            model_name = input("\n Enter the new model name (phi3 or llama or gemma): ")
            llm = load_llm(model_name)
            QA_LLM = retrieve_docs(embeddings, llm)

# Phi3 Responses

In [ ]:
if __name__ == "__main__":
    data_path = "/kaggle/input/axisfaq-data"
    documents = load_all_files(data_path)
    texts = interpret_files(documents)
    embeddings = create_embeddings()
    save(texts, embeddings)

    # model_path = "D:\Axis-FAQ-chatbot\models\llama-2-7b-chat.ggmlv3.q8_0.bin"
    model_name = input("Enter the model name (phi3 or llama or gemma): ")
    llm = load_llm(model_name)
    QA_LLM = retrieve_docs(embeddings, llm)

#     user_input = input("What is your question? \n")
#     answer = answer_question(QA_LLM, user_input)
#     print(f"Answer: {answer}")

    while True:
        user_input = input("\n What is your question? \n")
        if user_input.lower() == "quit":
            break
        answer = answer_question(QA_LLM, user_input)
        print(f"Answer: {answer}")

        continue_input = input("\n Do you want to ask another question? (yes to continue, quit to exit, change to switch model): ").strip().lower()
        if continue_input == "quit":
            break
        elif continue_input == "change":
            model_name = input("\n Enter the new model name (phi3 or llama or gemma): ")
            llm = load_llm(model_name)
            QA_LLM = retrieve_docs(embeddings, llm)